# Attention is all you need(NIPS 2017)

- 어텐션: 쿼리 행렬과 키 행렬에 행렬곱을 수행한 후 소프트맥스를 취해서 value 행렬에 적용할 웨이트를 구하는 것이 행렬곱 어텐션의 기본 구조이다.

- 쿼리 행렬과 키 행렬을 곱함으로써 둘 사이의 연관성을 파악하게 된다.

- 이번에는 2021년 기준 현대의 자연어 처리 네트워크에서 가장 핵심이 되는 모델 중 하나인 `트랜스포머(Transformer)`에 대해 실습해보려고 한다. 
- 트랜스포머는 기존의 `seq2seq` 모델과 동일하게 인코더와 디코더 구조를 사용한다.
 그러나 인코더와 디코더에서 RNN이나 CNN을 전혀 사용하지 않고, Attention 과정을 여러번 반복한다는 특징이 있다.
- 이를 통해 연산 횟수를 기존의 `seq2seq` 모델보다 확연히 줄이고, 병렬 연산의 정도도 늘어나 학습 시간도 크게 단축할 수 있었다고 한다.
이번에는 2021년 기준 현대의 자연어 처리 네트워크에서 가장 핵심이 되는 모델 중 하나인 `트랜스포머(Transformer)`에 대해 실습해보려고 한다.
- 트랜스포머는 기존의 `seq2seq` 모델과 동일하게 인코더와 디코더 구조를 사용한다.
- 그러나 인코더와 디코더에서 RNN이나 CNN을 전혀 사용하지 않고, Attention 과정을 여러번 반복한다는 특징이 있다.
- 이를 통해 연산 횟수를 기존의 `seq2seq` 모델보다 확연히 줄이고, 병렬 연산의 정도도 늘어나 학습 시간도 크게 단축할 수 있었다고 한다.
- 현재 자연어 처리에서 가장 최신 네크워크 중 하나인 `BERT`도 트랜스포머의 구조를 채택하고 있다.
- 아래의 링크를 참조하여 트랜스포머를 구현해보았다.  
[paper: Attention is all you need(NIPS 2017)](https://arxiv.org/pdf/1706.03762.pdf)  
[Seq2Seq with attention + Transformer review](https://www.youtube.com/watch?v=AA621UofTUA)  
[Tensorflow tutorial](https://www.tensorflow.org/tutorials/text/transformer)

- 트랜스포머의 가장 핵심 아이디어는 `self-attention`이다.
- 문장 번역을 예로 들면, `self-attention`을 이용하여 입력 문장의 단어들이 서로에 대한 연관성을 파악하여 해당 문장의 표현(representation of that sequence)을 구할 수 있다.
- 트랜스포머는 이러한 `self-attention` 층을 여러 층 쌓는 방식으로 모델을 생성하였다.

- 트랜스포머는 RNN이나 CNN을 사용하지 않고도 self-attention을 여러 층 사용하는 방식으로 variable-sized input을 처리할 수 있는데, 이러한 아키텍쳐는 아래와 같은 이점이 있다.
    - 데이터 전체에 걸쳐 시/공간적 관계를 가정하지 않는데 이는 a set of objects를 처리하는데 매우 적합하다.
    - RNN의 경우 입력 sequence를 순차적으로 입력해야 하는데, 트랜스포머의 경우 이를 병렬적으로 한번에 입력할 수 있다.
    - 멀리 떨어져 있는 값들(distant items)이 RNN이나 CNN의 통과 없이도 서로의 출력에 영향을 미칠 수 있다.
    - 따라서 트랜스포머는 long-range dependencies를 학습할 수 있다.
- 트랜스포머 아키텍쳐에는 이점만 있는 것이 아니라 다음과 같은 단점도 있다.
    - time-series 데이터의 경우 RNN을 사용할 때에는 각각의 타임 스텝마다의 출력이 현재의 입력과 hidden-state에 의해 계산되었지만, 트랜스포머에서는 전체 기록(entire history)가 구해져야 계산될 수 있는데 이는 비효율적일 수 있다.
    - 텍스트와 같이 데이터에 시/공간적 관계가 있는 경우 데이터에 이러한 관계를 넣어줄 수 있는 positional encoding을 추가하거나 bag of words를 사용하여야 한다.

In [39]:
### Setup
# $ pip install -q tensorflow_datasets
# $ pip install -q tensorflow_text

In [40]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [41]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)

### 데이터셋 다운로드
- tensorflow_datasets를 사용하여 `TED Talks Open Translation Project`의 포르투갈어-영어 번역 데이터셋을 다운로드한다.
- 출력을 보면, train_set에는 51785개의 문장이, validation_set에는 1193개의 문장이 포함되어 있다.

In [42]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)

In [43]:
train_examples, val_examples = examples['train'], examples['validation']
len(train_examples), len(val_examples)

(51785, 1193)

- tensorflow_datasets에 의해 리턴된 `tf.data.Dataset` 객체는 포트투갈어-영어의 텍스트 쌍을 생성한다.

In [44]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
    for pt in pt_examples.numpy():
        print(pt.decode('utf-8'))
    print()
    for en in en_examples.numpy():
        print(en.decode('utf-8'))

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
mas e se estes fatores fossem ativos ?
mas eles não tinham a curiosidade de me testar .

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


### 텍스트 tokenization & detokenization
- 데이터셋의 문장들을 문자 형태 그대로 인공신경망을 학습시킬 수는 없으므로 이를 먼저 인공신경망이 이해할 수 있는 숫자 데이터(numeric representation)로 변환하여야 한다.
- 일반적으로는 이를 위해 하나의 단어가 하나의 token ID를 가지도록 하여 각 문장을 sequences of token IDs로 변환한다.
- 이 token ID는 임베딩을 할 때 단어에 대한 인덱스로 사용된다.
- 문장을 tokenize하는 방법에는 `Subword tokenizer tutorial`의 내용에 따라 이번에 사용하는 데이터셋에 최적화된 subword tokenizers인 text.BertTokenizer를 빌드하고 이를 saved_model로 export하여 사용하는 방법이 있다.
- 아래의 코드는 tokenizer를 다운로드하고 saved_model로 불러오는 코드이다.

In [45]:
model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f"https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip",
    cache_dir='.', cache_subdir='', extract=True
)

'.\\./ted_hrlr_translate_pt_en_converter.zip'

In [46]:
tokenizers = tf.saved_model.load(model_name)

- tokenizers.en에는 아래와 같은 메서드를 포함하고 있다.

In [47]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

- 위에서 `tokenize` 메서드는 문장 배치를 토큰 ID로 이루어진 padded-batch로 변환해준다.
- 또한 토큰화하기 전에 구두점(punctuation)을 분할하고 입력된 문장을 unicode로 normalize한다.
- 이번 실습에서는 입력 문장이 이미 standardize되어 있으므로 이러한 standardization은 볼 수 없다.

In [48]:
for en in en_examples.numpy():
    print(en.decode('utf-8'))

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


In [49]:
encoded = tokenizers.en.tokenize(en_examples)

InvalidArgumentError:  Trying to access resource using the wrong type. Expected class tensorflow::lookup::LookupInterface got class tensorflow::lookup::LookupInterface
	 [[{{node StatefulPartitionedCall/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets}}]] [Op:__inference_restored_function_body_6879]

Function call stack:
restored_function_body
